In [0]:
%sql
USE CATALOG etl_project;
USE SCHEMA bigquery_db_cohort_db;

CREATE OR REPLACE TABLE silver_customer_facts AS
WITH ordered AS (
    SELECT
        customer_id,
        order_id,
        order_date,
        ROW_NUMBER() OVER (
            PARTITION BY customer_id
            ORDER BY order_date
        ) AS order_rank
    FROM bronze_ecom_orders
),

agg AS (
    SELECT
        customer_id,
        MIN(order_date) AS first_purchase_date,
        MIN(CASE WHEN order_rank = 2 THEN order_date END) AS second_purchase_date,
        COUNT(*) AS total_orders
    FROM ordered
    GROUP BY customer_id
)

SELECT
    customer_id,
    first_purchase_date,
    TO_DATE(DATE_TRUNC('month', first_purchase_date)) AS cohort_month,
    second_purchase_date,
    DATE_DIFF(second_purchase_date, first_purchase_date) AS days_between,
    DATE_DIFF(month, first_purchase_date, second_purchase_date) AS months_between,
    total_orders
FROM agg;
